<img src="../../thu_sigs_logo.png" alt="清华深研院-横" style="zoom:50%;" />

In [75]:
#| hide
#| default_exp hidden_markov_model.vis
# pdf:
#   toc: true
#   docx: default

In [1]:
#|hide
%load_ext autoreload
%autoreload 2
%load_ext rich

In [2]:
#|hide
from scholarly_infrastructure.logging.nucleus import logger, print
from thu_big_data_ml.help import plt, pio
import treescope

In [3]:
#|hide
treescope.active_autovisualizer.set_globally(treescope.ArrayAutovisualizer())

In [ ]:
#| hide 
import nbdev; nbdev.nbdev_export()

::: {.callout-important}
本文档具有一定的交互性，建议使用浏览器打开html文件，这样比pdf文件阅读体验更佳。
:::

## 第一题——一个例子理解 维特比 算法求解 HMM 模型的最优路径 {#sec-1}

> 给定盒子和球组成的隐马尔可夫模型$\lambda = (A, B, \pi)$，其中，
>
> $ A = \begin{bmatrix} 0.5 & 0.2 & 0.3 \\ 0.3 & 0.5 & 0.2 \\ 0.2 & 0.3 & 0.5 \end{bmatrix}, B = \begin{bmatrix} 0.5 & 0.5 \\ 0.4 & 0.6 \\ 0.7 & 0.3 \end{bmatrix}, \pi = (0.2, 0.4, 0.4)^T $
>
> 试用维特比算法求最优路径$I^* = (i_1^*, i_2^*, i_3^*, i_4^*)$。

要用维特比求出I，我们还需要知道观察序列$O = (o_1, o_2, o_3, o_4)$。
这是数据实在 李航书的 习题 10.1

> T=4, O=(红,白,红,白)



### 审题 {#sec-1-analysis}

我们首先复习一下李航书上的内容和课件上的内容。

- 隐马尔可夫模型的模型是什么？
- 隐马尔可夫模型的三个基本问题是什么？
- 除了上面三个问题，如果HMM用于时序预测又是怎么做的？
- HMM和EM算法是怎么关联起来的？

复习之后，我们来看看题目询问的问题。本题需要我们尝试和李航书例题解答不同的初值来尝试求解。

首先我们有了隐马尔可夫模型的三元组，可以定义一个马尔可夫模型

In [7]:
#| exports
import torch.nn as nn
import torch
from fastcore.all import patch, store_attr

In [8]:
#| exports
class HiddenMarkovModel(nn.Module):
    """Hidden Markov Model

    HMM with 3 states and 2 observation categories.

    Attributes:
        ob_category (list, with length 2): observation categories
        total_states (int): number of states, default=3
        pi (array, with shape (3,)): initial state probability
        A (array, with shape (3, 3)): transition probability. A.sum(axis=1) must be all ones.
                                      A[i, j] means transition prob from state i to state j.
                                      A.T[i, j] means transition prob from state j to state i.
        B (array, with shape (3, 2)): emitting probability, B.sum(axis=1) must be all ones.
                                      B[i, k] means emitting prob from state i to observation k.

    """
    def __init__(self):
        super(HiddenMarkovModel, self).__init__()
        self.ob_category = ['红球', '白球']  # 盒子里有红球白球两种球
        self.total_states = 3 # 有三个盒子
        self.pi = nn.Parameter(torch.Tensor([0.2, 0.4, 0.4]))
        self.A = nn.Parameter(torch.Tensor(
                            [[0.5, 0.2, 0.3],
                            [0.3, 0.5, 0.2],
                            [0.2, 0.3, 0.5]]))
        self.B = nn.Parameter(torch.Tensor(
                            [[0.5, 0.5],
                           [0.4, 0.6],
                           [0.7, 0.3]]))

In [9]:
model = HiddenMarkovModel()
treescope.show(model)

TreescopeFigure(
    treescope_part=PreventWordWrap(
        child=Siblings(
            children=(
                WithDynamicSharedPip(
                    child=FoldableTreeNodeImpl(
                        contents=Siblings(
                            children=(
                                IndentedChildren(
                                    children=[
                                        Siblings(
                                            children=(
                                                FoldableTreeNodeImpl(
                                                    contents=OnSeparateLines(
                                                        children=[
                                                            Siblings(
                                                                children=(
                                                                    Text(text='total_states='),
                                                                    NumberColor(child=Text(text='3')),
                                                                    Text(text=','),
                                                                    FoldCondition(
                                                                        collapsed=Text(text=' '),
                                                                        expanded=EmptyPart()
                                                                    ),
                                                                    FoldCondition(
                                                                        collapsed=EmptyPart(),
                                                                        expanded=Siblings(
                                                                            children=(
                                                                                StringCopyButton(
                                                                                    copy_string='.total_states',
                                                                                    annotation='Copy path: '
                                                                                ),
                                                                            )
                                                                        )
                                                                    )
                                                                )
                                                            ),
                                                            Siblings(
                                                                children=(
                                                                    Text(text='training='),
                                                                    KeywordColor(
                                                                        child=Text(text='True')
                                                                    ),
                                                                    Text(text=','),
                                                                    FoldCondition(
                                                                        collapsed=Text(text=' '),
                                                                        expanded=EmptyPart()
                                                                    ),
                                                                    FoldCondition(
                                                                        collapsed=EmptyPart(),
                                                                        expanded=Siblings(
                                                                            children=(
                                                                                StringCopyButton(
                                                               

In [19]:
# 而观测值为
observations = torch.Tensor([0, 1, 0, 1]).to(torch.int32)  # 红球，白球，红球，白球]

接下来尝试使用 viterbi 算法来解决 HMM 解码问题，我们要求出给定O，使得 P(I|O) 最大的 I 序列。

### 解题 {#sec-1-solution}



#### 解法一：使用Python自行编程实现Viterbi算法（学习使用）

注意，本代码部分参考了 https://github.com/thuhcsi/dpss-exp2-HMM/blob/master/T1-Viterbi-Decoding/viterbi.py 的实现。

In [16]:
#| exports
@patch
def decode_states_by_viterbi(self:HiddenMarkovModel, ob):
    """Viterbi Decoding Algorithm.

    Args:
        ob (array, with shape(T,)): (o1, o2, ..., oT), observations

    Variables:
        delta (array, with shape(T, 3)): delta[t, s] means max probability torwards state s at
                                            timestep t given the observation ob[0:t+1]
                                            给定观察ob[0:t+1]情况下t时刻到达状态s的概率最大的路径的概率
        phi (array, with shape(T, 3)): phi[t, s] means prior state s' for delta[t, s]
                                        给定观察ob[0:t+1]情况下t时刻到达状态s的概率最大的路径的t-1时刻的状态s'

    Returns:
        best_prob: the probability of the best state sequence
        best_path: the best state sequence

    """
    T = ob.shape[0]
    delta = torch.zeros((T, self.total_states))
    #update np.int32
    phi = torch.zeros((T, self.total_states), dtype=torch.int32)
    best_prob, best_path = 0.0, torch.zeros(T, dtype=torch.int32)

    # Begin Assignment

    # 从初始状态开始
    delta[0, :] = (self.pi * self.B[:, ob[0]])

    # 根据动态规划的公式来更新delta和phi
    for t in range(1, T):
        for j in range(self.total_states):
            delta[t, j], phi[t, j]= max((delta[t - 1, i] * self.A[i, j] * self.B[j, ob[t]], i) for i in range(self.total_states))

    # End Assignment

    best_path[T-1] = (delta[T-1, :].argmax(0))
    best_prob = delta[T-1, best_path[T-1]]
    for t in reversed(range(T-1)):
        best_path[t] = (phi[t+1, best_path[t+1]])

    return best_prob, best_path

In [21]:
best_prob, best_path = model.decode_states_by_viterbi(observations)
best_prob, best_path

(tensor(0.0030, grad_fn=<SelectBackward0>), tensor([2, 1, 1, 1], dtype=torch.int32))

我们三个盒子编号分别是0,1,2 ，根据代码输出的结果，最优的路径I是 2, 1, 1, 1。

#### 解法二：使用机器学习库（科研使用）


首先我们需要调查，哪些Python库可以实现HMM模型，并且支持解码问题。

1. 如果把HMM看做是机器学习模型，对应的机器学习库有没有支持HMM的呢？

我们最熟悉的sklearn库曾经是有HMM模型的，然而根据其[文档](https://scikit-learn.sourceforge.net/stable/modules/hmm.html)，由于HMM模型和其他sklearn模型的API有所不同，为了让项目更加专注，曾经的代码迁移到了 hmmlearn 库下，原本的sklearn已经移除了对应的支持，新库的链接为 [hmmlearn](https://github.com/hmmlearn/hmmlearn)。然而我们注意到，这个库并不是很活跃，与 [seqlearn](https://github.com/larsmans/seqlearn) 一样已经很久没有更新了。

2. 我们如果把HMM看做是时间序列的模型，那么对应的库有没有支持HMM的呢？

我们可以考虑使用 [sktime库](https://github.com/sktime/sktime), 这个库明显更加活跃，最新的更新在一天之内。sktime在时序模型上对sklearn的扩展，支持多种时序任务。

本题我们的任务不是 Forecasting，而是 sktime中称作 [Annotation](https://sktime-backup.readthedocs.io/en/v0.28.1/api_reference/auto_generated/sktime.annotation.hmm.HMM.html) 或者 [Detection](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.annotation.hmm.HMM.html) 的任务，在李航书里面叫做Tagging。 这些不同的叫法都是一个意思，找到已知模型参数和观测序列下，最大概率的状态序列。

另一个达到SOTA时间序列性能的库 AutoGluon Time Series, [支持了很多模型](https://auto.gluon.ai/dev/tutorials/timeseries/forecasting-model-zoo.html)，然而并不支持HMM模型。这个库主要的focus在于时序预测forecast的性能，因而并没有支持HMM。

3. 如果把HMM看做是简单的深度学习模型，深度学习库是否支持HMM？

那么我们最喜欢的PyTorch库，是否支持GPU上的HMM模型呢？看起来，确实是可以实现的，不过这些库主要是出于教育目的。
- [Hidden Markov Model in PyTorch](https://github.com/nwams/Hidden_Markov_Model-in-PyTorch/blob/master/Hidden%20Markov%20Models%20in%20PyTorch.ipynb)
- [pytorch_HMM](https://github.com/lorenlugosch/pytorch_HMM)

4. 如果把HMM看做是贝叶斯网络，[一个概率图模型](https://github.com/lorenlugosch/pytorch_HMM)，一个EM算法的使用案例，那么是否有做这方面的库？

答案是肯定的。首先我们了解一下，概率建模有关的Python库有哪些？

- Pyro：基于Python和PyTorch的概率编程库，采用随机变分推理技术，适用于处理复杂的模型和大规模数据集。

- PyMC3：Python包用于贝叶斯统计建模，专注于高级MCMC和变分推断算法，适用于一系列问题。

- Edward：由Google开发的基于TensorFlow的概率编程库，用于概率建模和不确定性量化。

- Stan：开源项目用于贝叶斯统计建模和概率编程，允许用户编写模型，并使用MCMC等方法进行拟合。

- Pomegranate：功能强大且易于使用的概率建模工具包，支持多种模型，包括隐马尔可夫模型。

- pgmpy：这是一个Python库，用于构建和操作概率图模型（PGMs），包括贝叶斯网络和马尔可夫随机场。它提供参数学习、推理（包括精确和近似推理）以及模型可视化等功能。

- pyAgrum：pyAgrum是C++库aGrUM的Python接口，专注于贝叶斯网络和其他概率图模型。它支持创建、建模、学习、使用、计算和嵌入贝叶斯网络等任务，同时提供了一些特有的Python和C++代码以简化和扩展aGrUM API。

- Dynamax：这是一个允许轻松定义状态空间模型的库。使用Dynamax，可以进行最大似然估计或使用Blackjax进行MCMC全贝叶斯后验估计。

- Blackjax：这是一个基于JAX的概率库，用于贝叶斯推断，特别是使用哈密顿蒙特卡洛（HMC）或序贯蒙特卡洛（SMC）方法计算参数后验。

以上的库大多数都支持HMM模型。

<!-- jax是个Google开发的现代基础张量计算库，速度相比numpy和PyTorch都非常快。我们来尝试 Dynamax。

```python
pip install dynamax
``` -->


## 第二题——推导公式 {#sec-2}

题目如下
> 试用前向概率和后向概率推导  
> 
> $P(O|\lambda)=\sum_{i=1}^{N}\sum_{j=1}^{N}\alpha_t(i)a_{ij}b_j(o_{t+1})\beta_{t+1}(j), \quad t=1,2,\cdots,T-1$


### 审题


- 我们需要理解题目给的式子中符号的含义。前向概率和后向概率的定义是什么？
- 李航给出的这个式子的意义是什么？

首先我们来明确一下，这道题目提出的问题还是概率计算问题，也就是说给定隐马尔可夫模型（HMM）$\lambda=(A,B,pi)$，观察序列 $O=(o_1,o_2,...,o_T)$，我们想要求模型参数生成了这个观察序列的概率 $P(O|\lambda)$。

我们首先来复习一下前向概率和后向概率的定义。

如果我们没有对观察序列是一个可以拆开的序列的理解，只是把 O, I 当做一个整体去看，李航首先在书中给出了暴力的（概念上可行但是计算上不可行的）直接计算法 [@LiHang_2022] ，Rabiner论文叫做"straightforward way" [@Rabiner_1989]。而根据进一步的观察，可以导出这个问题的两个动态规划算法，分别是前向算法和后向算法。

**而李航在这道题目似乎是给出了一个新的算法，又使用前向概率，又使用后向概率！**

::: {.callout-note}
注意，深度学习中反向传播求梯度算法与之不同，Hinton推广的反向传播算法确实也是节省计算量的算法，但是那个里面模型需要前向传播也需要反向传播，都要计算，而我们这里前向算法和后向算法是两个独立的、都可以解决问题的算法，尽管李航 [@LiHang_2022] 和 Rabiner [@Rabiner_1989] 都不恰当的合在一起命名为 "前向-后向算法"和"The Forward-Backward Procedure"，让人误解。

特别提到这个问题时因为最近Hinton又提出了所谓的 "Forward-Forward Algorithm"，这里面就是两个Forward都要做。所以我们命名方法的时候一定要精确。
::: 

#### 前向算法的复习 

前向算法是典型的动态规划思路 （不只是维特比算法是动态规划）。
首先我求 $P(O|\lambda)$ 可以有多种加和方法，不一定是从 $P(O, I|\lambda)$ 边缘化得到的。我们可以考虑所谓的前向概率，即 $P(O_{1:t}, I_t = q_i | \lambda)$ ，其中O_{1:t}的冒号是索引表达，而P(X=x)表示随机变量X取一个值x的概率密度。 这个概率记作 $a_{t}(i)$ ，是t和i的函数, 其中 $q_i$ 表示第i个状态。

如果我求出了 $P(O_{1:T}, I_T = q_i | \lambda)$ 所有i下的情况，那我就能求出 $P(O|\lambda)$ 。

而 $P(O_{1:T}, I_T = q_i | \lambda)$ 本来也是不好求的。因为我们可以看着贝叶斯网络 @fig-hmm-demo, 可以发现我只知道 $P(O_t | I_t)$ 和 $P(I_t | I_{t-1})$ 就算你帮我减少到只有 $I_T = q_i$, $O_T$自然是和前面大家无关了, $O_{1:T}$ 这里还有 T-1个独立的rv需要遍历所有情况。

我们首先通过贝叶斯网络的观察，化简了问题为 $P(O_{1:T}, I_T = q_i | \lambda) = P(O_{1:T-1}, I_T = q_i | \lambda) \times P( O_T | I_T = q_i, \lambda) $，但是接下来$P(O_{1:T-1}, I_T = q_i | \lambda)$要怎么办呢？

这个时候，我们将问题拆解为重叠的子问题，如果我知道 $P(O_{1:T-1}, I_{T-1} = q_j | \lambda)$ 所有的j的情况呢？

想到了这个，我们恍然大悟，只需要再来一个A矩阵，就可以用求和公式和贝叶斯公式得到 $P(O_{1:T-1}, I_T = q_i | \lambda) = [\sum_{j=1}^N P(O_{1:t-1}, I_{t-1} = q_j | \lambda) \cdot a_{ji}]$ 。
<!-- TODO  -->

所以

$P(O_{1:t}, I_T = q_i | \lambda) = [\sum_{j=1}^N P(O_{1:t-1}, I_{t-1} = q_j | \lambda) \cdot a_{ji}] \times b_i(O_t)$

至此，我们就推导出了前向算法，按照李航书上的符号总结为算法步骤如下:

$$
\boxed{
\begin{aligned}
&\text{输入：隐马尔可夫模型 } \lambda, \text{ 观测序列 } O. \\
&\text{输出：观测序列概率 } P(O|\lambda). \\
&\text{(1) 初值} \\
&\alpha_1(i) = \pi_i b_i(o_1), \quad i = 1, 2, \ldots, N \quad  \\
&\text{(2) 递推} \\
&\text{对于 } t = 1, 2, \ldots, T - 1: \\
&\alpha_{t+1}(i) = \left[ \sum_{j=1}^{N} \alpha_t(j) a_{ji} \right] b_i(o_{t+1}), \quad i = 1, 2, \ldots, N \quad  \\
&\text{(3) 终止} \\
&P(O|\lambda) = \sum_{i=1}^{N} \alpha_T(i) \quad 
\end{aligned}
}
$$

前向算法运用广泛，Zhang在论文中就采用了前向算法来计算HMM的概率 [@Zhang_2024]。

#### 后向算法的复习 

后向算法的思路与前向算法基本一致，只是递推的方向反过来，最后会尝试从 i_1 去求和得到 $P(O|\lambda)$ 。前向算法是从前到后计算$\alpha_t(i)$，而后向算法是从后往前计算$\beta_t(i)$。

首先我们按照李航书上对后向概率 $\beta$ 进行定义 [@LiHang_2022]：

> $$
> \beta_t(i) = P(o_{t+1}, o_{t+2}, \cdots, o_T \mid i_t = q_i, \lambda)
> $$

也就是时刻t位于某个状态$q_i$的条件下，观测序列的后续部分的概率。

这里的关键还是递推。从时间$T-1$向前递推，计算每个时间$t$的$\beta_t(i)$。在时间$t$，如果模型处于状态$i$，那么接下来观测到$o_{t+1}$的概率取决于从状态$i$转移到所有可能的下一个状态$j$的概率$a_{ij}$，以及在状态$j$下观测到$o_{t+1}$的概率$b_j(o_{t+1})$，再乘以下一个时间点的$\beta_{t+1}(j)$。

因此，递推公式为：

$$
\beta_t(i) = \sum_{j=1}^{N} a_{ij} b_j(o_{t+1}) \beta_{t+1}(j), \quad \forall i = 1, 2, \ldots, N; \ t = T-1, T-2, \ldots, 1 \quad 
$$

整体的算法如下

$$
\boxed{
\begin{aligned}
&\text{输入：隐马尔可夫模型 } \lambda, \text{ 观测序列 } O. \\
&\text{输出：观测序列概率 } P(O|\lambda). \\
&\text{(1) 初始化} \\
&\beta_T(i) = 1, \quad \forall i = 1, 2, \ldots, N \quad  \\
&\text{(2) 递推} \\
&\text{对于 } t = T-1, T-2, \ldots, 1: \\
&\beta_t(i) = \sum_{j=1}^{N} a_{ij} b_j(o_{t+1}) \beta_{t+1}(j), \quad \forall i = 1, 2, \ldots, N \quad  \\
&\text{(3) 终止} \\
&P(O|\lambda) = \sum_{i=1}^{N} \pi_i b_i(o_1) \beta_1(i) \quad 
\end{aligned}
}
$$


### 解题 {#sec-2-solution}


我们首先来看题目所给的式子

> $P(O|\lambda)=\sum_{i=1}^{N}\sum_{j=1}^{N}\alpha_t(i)a_{ij}b_j(o_{t+1})\beta_{t+1}(j), \quad t=1,2,\cdots,T-1$

其中 
$\beta_t(i) = P(o_{t+1}, o_{t+2}, \cdots, o_T \mid i_t = q_i, \lambda)$, $\alpha_t(i) = P(o_1, o_2, \cdots, o_t, i_t = q_i | \lambda)$

题目式子是说，如果我们考虑任何一个时间点t和下一个时间点t+1, t时刻的状态设为$q_i$, t+1时刻的状态设为$q_j$。首先，发生这个转移的状态转移的概率是 $a_ij$，这很好理解。从t+1的状态导出的 $o_{t+1}$的概率是 $b_j(o_{t+1})$，这也很好理解。

题目想问的是，如果我们这个时候把 t的前向概率和t+1的后向概率拼接在一起，能不能得到整一个O的概率？答案是肯定的。我们使用概率公式进行推导证明这个公式。

--- 


所谓证明，我们要不是从左边的式子展开得到右边的式子，要不是从右边的式子合并得到左边的式子。

这里我们选择从左边的式子展开得到右边的式子。我们可以将$P(O|\lambda)$在t和t+1的位置展开。

根据全概率公式，我们可以将$P(O|\lambda)$表示为所有可能的状态序列的概率之和：

$$
P(O|\lambda) = \sum_{i=1}^{N} \sum_{j=1}^{N} P(o_1, \ldots, o_t, i_t = q_i, i_{t+1} = q_j, o_{t+1}, \ldots, o_T | \lambda)
$$

这里，我们引入了中间状态变量$i_t = q_i$和$i_{t+1} = q_j$。

注意李航的书上符号稍微有些混乱，不要把下标i和状态序列i搞混了。

这里我们已经初步看到了右边式子求和的结构。

---


接下来，我们只需要证明这个联合概率可以分解成已知的变量：

$$
P(o_1, \ldots, o_t, i_t = q_i, i_{t+1} = q_j, o_{t+1}, \ldots, o_T | \lambda) = \alpha_t(i) \cdot a_{ij} \cdot b_j(o_{t+1}) \cdot \beta_{t+1}(j)
$$

要想证明这个，我们补充一下贝叶斯网络的知识 [@Zhou_2016]。

贝叶斯网络展示了这些变量之间的依赖关系。重要的是：

1. 每个隐状态 $ i_{t} $ 只依赖于前一个隐状态 $ i_{t-1} $。
2. 每个观测 $ o_t $ 只依赖于对应的隐状态 $ i_t $.

因此，贝叶斯网络的依赖结构如下：

$$ i_1 \rightarrow i_2 \rightarrow \ldots \rightarrow i_T $$
$$ i_1 \rightarrow o_1, \quad i_2 \rightarrow o_2, \ldots, i_T \rightarrow o_T $$


根据贝叶斯网络的概率的链式法则，联合概率可以分解，每一个联合概率里面出现的rv，只需要考虑自己的父节点作为条件，而不用考虑其他节点的影响，最后直接乘在一起。因此，我们可以将联合概率分解为如下形式：

$$
P(o_1, \ldots, o_t, i_t = q_i, i_{t+1} = q_j, o_{t+1}, \ldots, o_T | \lambda) = P(o_1, \ldots, o_t, i_t = q_i | \lambda) 
\\ \cdot P(i_{t+1} = q_j | i_t = q_i, \lambda) 
\\ \cdot P(o_{t+1}, \ldots, o_T | i_{t+1} = q_j, \lambda)
$$

很多同学推导到这里就懵了，这里我们只有三个式子，原本有四个式子呀，去哪了？

原来，因为 $\beta_t(i) = P(o_{t+1}, o_{t+2}, \cdots, o_T \mid i_t = q_i, \lambda)$ 
也就是说 $\beta_{t+1}(i) = P(o_{t+2}, o_{t+3}, \cdots, o_T \mid i_{t+1} = q_i, \lambda)$ 

而这里面的 第三项乘积项是 $P(o_{t+1}, \ldots, o_T | i_{t+1} = q_j, \lambda)$

正是因为这个差别，所以多出了 $b_j(o_{t+1})$, 因为

$$
P(o_{t+1}, \ldots, o_T | i_{t+1} = q_j, \lambda) = P(o_{t+2}, o_{t+3}, \cdots, o_T \mid i_{t+1} = q_j, \lambda) \times P(o_{t+1} \mid i_{t+1} = q_j, \lambda)
$$

至此我们就证明清楚了


$$
P(o_1, \ldots, o_t, i_t = q_i, i_{t+1} = q_j, o_{t+1}, \ldots, o_T | \lambda) = \alpha_t(i) \cdot a_{ij} \cdot b_j(o_{t+1}) \cdot \beta_{t+1}(j)
$$



--- 


结合以上结果, 把 $P(o_1, \ldots, o_t, i_t = q_i, i_{t+1} = q_j, o_{t+1}, \ldots, o_T | \lambda) = \alpha_t(i) \cdot a_{ij} \cdot b_j(o_{t+1}) \cdot \beta_{t+1}(j)$ 代入到全概率公式里面，那么

$$
P(O|\lambda) = \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_t(i) \cdot a_{ij} \cdot b_j(o_{t+1}) \cdot \beta_{t+1}(j)
$$


这个公式表明，在每个时间点$t$，观测序列的概率可以通过前向变量$\alpha_t(i)$、状态转移概率$a_{ij}$、观测概率$b_j(o_{t+1})$和后向变量$\beta_{t+1}(j)$的结合来计算。


### 题目扩展问题





In [74]:
#| hide
import nbdev; nbdev.nbdev_export()